In [18]:
from pyomo.environ import *
from pyomo.environ import Binary, NonNegativeReals
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple

In [19]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 52  # Number of execution scenarios                     (chi)

M = 1e3  # Big M

all_scenario_names = ["good", "avg_plus", "average", "avg_minus", "bad"]
ns_dict = {'good': 52, "average": 26, "bad":0, "avg_plus":30, "avg_minus":13}
scenario_probabilities = {'good': 0.2, 'average': 0.2, 'bad': 0.2, "avg_plus":0.2, "avg_minus":0.2}

In [20]:
def build_model(cap_factor):
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=125, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    # com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 125
    
    prod_max = {0:0.25*com1_process_capacity, 1: 0.5*com1_process_capacity, 2: 0.75*com1_process_capacity, 3: 0.95*com1_process_capacity, 4: com1_process_capacity}
    prod_min = {0:0, 1: 0.25*com1_process_capacity, 2: 0.5*com1_process_capacity, 3: 0.75*com1_process_capacity, 4: 0.95*com1_process_capacity}
    rate_max = {0:1.25/2, 1: 1/2, 2: 0.75/2, 3: 0.5/2, 4: 0.25/2}
    # mode_ramp = {(0,1): 5, (1,2): 5}
    
    com1_procure = Process(name='procure com1', prod_max=com1_process_capacity, conversion={com1_pur: -1, com1_in: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Procure com1', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Sell com1')
    com1_opt_procure = Process(name='procure optional com1', prod_max=75, conversion={com1_pur: -1, com1_in:1}, capex=10, vopex=0.01, prod_min=0.01, label='Procure optional com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 1')
    # com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 2')
    
    # com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Process the commodity through the location')
    # com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}, 3:{com1_in: -1, com1_out: 1}, 4:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=0.01, label='Process the commodity through the location')
    # 
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.1, vopex=0.01, prod_min=0.01, label='Process the commodity through the location')
    
    com1_store10 = Process(name='com1_store10', prod_max=com1_process_capacity, capex=0.1, vopex=0.01, storage_capex=10, store_min=0.01, store_max= 40, prod_min=0.01, label="Storage capacity of 10 units", storage=com1_in, storage_cost=0.02)
    com1_store20 = Process(name='com1_store20', prod_max=com1_process_capacity, capex=0.1, vopex=0.02, storage_capex=20, store_min=0.01, store_max= 80, prod_min=0.01, label="Storage capacity of 20 units", storage=com1_in, storage_cost=0.02)
    com1_store50 = Process(name='com1_store50', prod_max=com1_process_capacity, capex=0.1, vopex=0.05, storage_capex=50, store_min=0.01, store_max= 200, prod_min=0.01, label="Storage capacity of 50 units", storage=com1_in, storage_cost=0.02)
    
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 1')
    # com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 2')
    
    
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_process, com1_store10, com1_store20, com1_store50, com1_loc1_send, com1_sell, com1_opt_procure}, label="Location 1", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_procure: cap_factor[['com1_procure']]})
    
   
    locset = [loc1]


    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 100
    demand_penalty = 20
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc1 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc1 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario_baseline', network= loc1, scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Scenario with perfect information')
    
    # ======================================================================================================================
    # Declare problem
    # ======================================================================================================================
    
    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    
    scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
    capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
    
    problem_mincost.first_stage_cost  = capex_process
    
    return scenario, problem_mincost

# Perfect Information

In [21]:
exCost_PI = 0

for scenario_name in all_scenario_names:
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen_PI, model_PI = build_model(cap_factor=cap_factor)
    # for loc in model.locations:
    #     model.X_P[loc,"procure com1",scen.network_scale_level].fixed= True
    #     model.X_P[loc,"procure com1",scen.network_scale_level].value= 1
    # solver = SolverFactory("gurobi")
    # solver.solve(model_PI)
    
    results_good = solve(scenario=scen_PI, instance=model_PI, solver='gurobi', name=scenario_name)
    
    # print(f"################## OBJECTIVE VALUE: {value(model_PI.objective_cost_w_demand_penalty)} ##################")
    # print(f"################## NETWORK CAPEX: {value(model_PI.Capex_network[0])} ##################")
    # print(f"################## NETWORK VOPEX: {value(model_PI.Vopex_network[0])} ##################")
    # print(f"################## NETWORK InvHold: {value(model_PI.Inv_cost_network[0])} ##################")
    # print(f"################## DEMAND PENALTY: {value(model_PI.Demand_penalty_cost_network['com1_sold',0])} ##################")
    # 
    # model_PI.write('model_PI_'+scenario_name+'.lp')
    # 
    # with open('model_PI_'+scenario_name+'.lp', 'w') as f:
    #     for v in model_PI.component_objects(Var, active=True):
    #         varobject = getattr(model_PI, str(v))
    #         for index in varobject:
    #             f.write(f'{varobject[index].name}: {varobject[index].value}\n')
    
    exCost_PI += value(model_PI.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
constraint demand penalty location
constraint demand penalty network
constraint demand penalty cost
constraint demand penalty cost location
constraint demand penalty cost network
objective cost w demand penalty
Set parameter QCPDual to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 13605 row

In [22]:
print(f"Expected cost under perfect information: {exCost_PI:.3f}")

Expected cost under perfect information: 98646.850


# Expected Value of Perfect Information

In [23]:
def scenario_creator(scenario_name):    
    cap_factor_scen = pandas.DataFrame(data={'com1_procure': [1] * ns_dict[scenario_name] + [0] * (_exec_scenarios - ns_dict[scenario_name])})
    scen, model = build_model(cap_factor=cap_factor_scen)
    sputils.attach_root_node(model, model.first_stage_cost, [model.X_P, model.Cap_P, model.X_S, model.Cap_S])
    model._mpisppy_probability = scenario_probabilities[scenario_name]
    return model

In [24]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "gurobi"}
# all_scenario_names = ["good", "average", "bad"]
ef_UI = ExtensiveForm(options, all_scenario_names, scenario_creator, model_name='model_UI')
results = ef_UI.solve_extensive_form()

[   73.53] Initializing SPBase
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
constraint demand penalty location
constraint demand penalty network
constraint demand penalty cost
constraint demand penalty cost location
constraint demand penalty cost network
objective cost w demand penalty
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
cons

In [25]:
exCost_UI = ef_UI.get_objective_value()
print(f"Expected cost under uncertainty: {exCost_UI:.3f}")

Expected cost under uncertainty: 98961.879


In [26]:
EVPI = exCost_UI - exCost_PI
EVPI

315.02943089429755

# Value of Stochastic Solution

In [27]:
n_avg = int(sum(ns_dict[i] for i in ns_dict.keys())/3)
cap_factor_avg = pandas.DataFrame(data={'com1_procure': [1]*n_avg + [0]*(_exec_scenarios-n_avg)})
scen_avg, model_avg = build_model(cap_factor=cap_factor_avg)

solver_avg = SolverFactory("gurobi")
solver_avg.solve(model_avg)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
constraint demand penalty location
constraint demand penalty network
constraint demand penalty cost
constraint demand penalty cost location
constraint demand penalty cost network
objective cost w demand penalty


{'Problem': [{'Name': 'x1', 'Lower bound': 43431.99999999998, 'Upper bound': 43432.0, 'Number of objectives': 1, 'Number of constraints': 13605, 'Number of variables': 11043, 'Number of binary variables': 14, 'Number of integer variables': 14, 'Number of continuous variables': 11029, 'Number of nonzeros': 38394, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.04699993133544922', 'Error rc': 0, 'Time': 0.2474656105041504}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [28]:
def fix_variables(model1: ConcreteModel, model2:ConcreteModel):
    
    def fix(var1, var2):
        for i in list(var1.keys()):
            var2[i].fixed = True
            var2[i] = value(var1[i])
    
    # def fix(var1, var2):
    #     for i in var1:
    #         if var1[i].value is not None:
    #             var2[i].fix(value(var1[i]))
            
    fix(model1.X_P, model2.X_P)
    fix(model1.Cap_P, model2.Cap_P)
    fix(model1.X_S, model2.X_S)
    fix(model1.Cap_S, model2.Cap_S)

In [29]:
exCost_FD = 0

for scenario_name in all_scenario_names:
    cap_factor_VSS = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen_VSS, model_VSS = build_model(cap_factor=cap_factor_VSS)

    fix_variables(model1=model_avg, model2=model_VSS)

    solver = SolverFactory("gurobi")
    solver.solve(model_VSS)

    # print(f"################## OBJECTIVE VALUE: {value(model_VSS.objective_cost_w_demand_penalty)} ##################")
    # print(f"################## NETWORK CAPEX: {value(model_VSS.Capex_network[0])} ##################")
    # print(f"################## NETWORK VOPEX: {value(model_VSS.Vopex_network[0])} ##################")
    # print(f"################## NETWORK InvHold: {value(model_VSS.Inv_cost_network[0])} ##################")
    # print(f"################## UNMET DEMAND: {value(model_VSS.Demand_penalty_network['com1_sold',0])} ##################")
    # print(f"################## DEMAND PENALTY COST: {value(model_VSS.Demand_penalty_cost_network['com1_sold',0])} ##################")

    exCost_FD += value(model_VSS.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
constraint demand penalty location
constraint demand penalty network
constraint demand penalty cost
constraint demand penalty cost location
constraint demand penalty cost network
objective cost w demand penalty
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constr

In [30]:
exCost_PI

98646.85

In [31]:
exCost_UI

98961.8794308943

In [32]:
exCost_FD

98962.65000000001

In [33]:
EVPI

315.02943089429755

In [34]:
VSS = exCost_FD - exCost_UI
VSS

0.7705691057053627